In [122]:
# Step to ensure that the venv is being used for the project not local copies, should point at .venv in project.
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv")) 

python: c:\work\Hackathon-2025\.venv\Scripts\python.exe
uv: c:\work\Hackathon-2025\.venv\Scripts\uv.EXE


In [123]:
# installs into the current Jupyter kernel environment
%pip install -U uv 
#! to run shell commands
!uv pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Using Python 3.13.7 environment at: c:\work\Hackathon-2025\.venv
Audited 13 packages in 35ms


In [124]:
# LangChain + MCP Setup for Attractions Booking (HTTP-based for Jupyter)
import os
from dotenv import load_dotenv
import asyncio
import json
import requests
from typing import Dict, Any, List

# LangChain imports
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

print("✅ Updated imports with official MCP adapter loaded successfully!")

✅ Updated imports with official MCP adapter loaded successfully!


In [125]:
# MCP Client Setup using Official Adapter with HTTP Transport
import subprocess
import time

# Global MCP client for HTTP
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Create MultiServerMCPClient with streamable_http transport
        mcp_client = MultiServerMCPClient({
            "courses": {
                "transport": "streamable_http",
                "url": os.getenv("COURSES_MCP_URL")
            },
            "hr": {
                "transport": "streamable_http",
                "url": os.getenv("HR_MCP_URL")
            }
        })
        
        # Get tools from the MCP server
        tools = await mcp_client.get_tools()
        print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
        return tools
        
    except Exception as e:
        print(f"Error connecting to MCP HTTP server: {e}")
        return []

print("🔗 MCP HTTP adapter setup ready!")

🔗 MCP HTTP adapter setup ready!


In [126]:
# Load MCP Tools using Official Adapter
# The tools will be loaded dynamically when setting up the agent
# No need to manually create tool wrappers - the adapter handles this automatically
print("🛠️ Ready to load MCP tools via official adapter!")

🛠️ Ready to load MCP tools via official adapter!


In [127]:
async def setup_agent():
    """Setup LangChain agent with MCP tools using official adapter"""
    
    # Initialize LLM for Azure OpenAI
    # can get this from Azure Open Ai service -> Azure Ai Foundary Portal
    from langchain_openai import AzureChatOpenAI
    
    llm = AzureChatOpenAI(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),  # Your Azure deployment name
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_version=os.getenv("AZURE_API_VERSION"), 
        temperature=1
    )
    
    # Load MCP tools using official adapter
    tools = await create_mcp_tools()
    
    if not tools:
        print("No MCP tools loaded. Make sure the MCP server is accessible.")
        return None
    
    print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
    
    # Create system prompt
    system_prompt = """You are an AI assistant specialising in helping to identify and book courses for employees based on their previous training.
    You have access to two integrated services that help provide information about courses and employees including the courses they've
    completed which allows the competencies to be derived.

    If you need to identify people then use the HR service, if you need to identify courses then use the Courses service.

    Courses have different levels (beginner, intermediate, advanced), where employees have completed higher level courses for particular skills
    then they should not be recommended lower level courses for those skills.

    Each course has a set of job roles that it is relevant for, if an employee's job role matches one of these then the course is more relevant. 
    If the employee's job role does not match then the course is less relevant and should NOT be suggested.
    If it is ambiguous if the job role is relevant for a course then suggest the course but explain the ambiguity allowing the user to make the final decision.

    You should always attempt to answer the users query in relation to employees rather than just generic groups of people or job roles.
"""

#     If you cannot find a suitable course for an employee then say so, do not try to suggest courses that are not returned by your tools.
# """
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create agent executor with tool logging callback and verbose output
    agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
    
    return agent_executor

# Initialize the agent (now async)
agent_executor = None
print("🤖 Agent setup function ready! Run the next cell to initialize.")

🤖 Agent setup function ready! Run the next cell to initialize.


In [128]:
# Initialize the agent with MCP tools
async def initialize_agent():
    """Initialize the agent with MCP tools"""
    global agent_executor
    print("Initializing agent with MCP tools...")
    agent_executor = await setup_agent()
    if agent_executor:
        print("LangChain agent with MCP tools ready!")
    else:
        print("Failed to initialize agent. Check MCP server connection.")

# Run the initialization
await initialize_agent()

Initializing agent with MCP tools...
Loaded 6 MCP tools: ['search_courses', 'get_course_details', 'search_and_format_courses', 'get_course_categories', 'get_courses_for_role', 'search_employees']
Loaded 6 MCP tools: ['search_courses', 'get_course_details', 'search_and_format_courses', 'get_course_categories', 'get_courses_for_role', 'search_employees']
LangChain agent with MCP tools ready!


In [129]:
# User Input Handler + logged agent steps
async def process_user_input(user_input: str) -> str:
    """Process user input and return LLM response using MCP tools"""
    if not agent_executor:
        return "Agent not initialized. Please run the initialization cell first."
    
    try:
        # Use the agent to process the input and get intermediate steps
        result = await agent_executor.ainvoke({"input": user_input})
        output = result.get("output") or result.get("final_output") or ""

        # Print intermediate steps if present
        steps = result.get("intermediate_steps") or []
        for step in steps:
            action = None
            observation = None
            if isinstance(step, tuple) and len(step) == 2:
                action, observation = step
            elif isinstance(step, dict) and "action" in step:
                action = step.get("action")
                observation = step.get("observation")
            else:
                continue

            tool_name = getattr(action, "tool", getattr(action, "tool_name", "unknown"))
            tool_args = getattr(action, "tool_input", getattr(action, "input", None))
            print(f"\n--- Tool: {tool_name}")
            print(f"args: {tool_args}")
            if observation is not None:
                print(f"result: {observation}")
            print("---\n")

        return output
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Interactive function for easy testing
async def ask_assistant(question: str):
    """Easy-to-use function for asking the travel assistant"""
    print(f"🧳 User: {question}")
    print("🤖 Assistant:")
    
    response = await process_user_input(question)
    print(response)
    return response

print("💬 User input handler ready!")

💬 User input handler ready!


# ‼️Important

Make sure you add the following lines to the .env file

```
COURSES_MCP_URL=http://127.0.0.1:8011/mcp/
HR_MCP_URL=http://127.0.0.1:8010/mcp/
```

In [134]:
# Test MCP server connectivity and tools
async def test_mcp_connection():
    """Test MCP server connection and list available tools"""
    tools = await create_mcp_tools()
    if tools:
        print(f"MCP HTTP server connected successfully!")
        print(f"Available tools: {[tool.name for tool in tools]}")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    else:
        print("Failed to connect to MCP HTTP server")

# Test MCP HTTP connection
await test_mcp_connection()


Loaded 6 MCP tools: ['search_courses', 'get_course_details', 'search_and_format_courses', 'get_course_categories', 'get_courses_for_role', 'search_employees']
MCP HTTP server connected successfully!
Available tools: ['search_courses', 'get_course_details', 'search_and_format_courses', 'get_course_categories', 'get_courses_for_role', 'search_employees']
  - search_courses: Search for IT training courses with optional filters

Args:
    category: Course category to filter by. Available categories:
              - programming (Python, Java, JavaScript, etc.)
              - cloud (AWS, Azure, GCP)
              - devops (Docker, Kubernetes, CI/CD)
              - data (Data Science, Machine Learning)
              - security (Cybersecurity, Ethical Hacking)
              - web (Frontend, Backend, Full-stack)
              - mobile (iOS, Android, React Native)
              - database (SQL, NoSQL)
              You can search multiple categories by separating with commas (e.g., "programmin

In [131]:
# 🚀 EXAMPLE USAGE - Run this cell after setting up your API key!

# Simple question
# await ask_assistant("Please provide me with a list of courses for the category 'programming'.")
# print("\n" + "="*50 + "\n")


In [132]:
# Interactive chat loop — keep asking questions until you exit

# try -
# Please give me a list of all courses


async def chat_loop():
    print("Type 'exit' to quit. Press Enter on an empty line to skip.")
    while True:
        try:
            question = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting.")
            break
        if not question:
            continue
        if question.lower() in ("exit", "quit", "q"):
            print("Goodbye!")
            break
        await ask_assistant(question)

# Start the loop
await chat_loop()

Type 'exit' to quit. Press Enter on an empty line to skip.
🧳 User: Joe Bloggs has a basic level of python code knowledge but would like to expand on this for his next project. Can you suggest any courses he should enrol on?
🤖 Assistant:


> Entering new AgentExecutor chain...

Invoking: `search_employees` with `{'name': 'Joe Bloggs'}`


{
  "total_count": 1,
  "employees": [
    {
      "id": 1011,
      "name": "Joe Bloggs",
      "email": "joe.bloggs@example.com",
      "phone": "",
      "department": "Software Engineering",
      "position": "Principal Developer",
      "completed_courses": [
        "prog_003"
      ]
    }
  ]
}
Invoking: `get_course_details` with `{'course_id': 'prog_003'}`


{
  "course": {
    "id": "prog_003",
    "title": "Java Spring Boot Microservices",
    "duration": "10 weeks",
    "start_date": "2025-03-01",
    "description": "Build enterprise-grade microservices with Java Spring Boot, Docker, and Kubernetes.",
    "category": "programming",
    "leve

In [133]:
# Cleanup function for HTTP MCP client
async def cleanup_mcp():
    """Cleanup MCP client and server resources"""
    global mcp_client
    if mcp_client:
        try:
            await mcp_client.close()
            print("MCP client closed")
        except Exception as e:
            print(f"Warning: Error closing MCP client: {e}")
        mcp_client = None

print("🧹 Cleanup function ready!")

🧹 Cleanup function ready!
